In [3]:
%load_ext autoreload
%autoreload 2

import os
import sys

# Build an absolute path from this notebook's parent directory
module_path = os.path.abspath("../")

# Add to sys.path if not already present
if module_path not in sys.path:
    sys.path.append(module_path)

import numpy as np
from aerosandbox import Airfoil, KulfanAirfoil
from src.Airfoil import airfoil_modifications

import plotly.graph_objects as go

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import matplotlib.pyplot as plt

def plot_samples(Z, airfoils, scale=0.8, points_per_axis=None, scatter=True, symm_axis=None, annotate=False, fname=None, **kwargs):
    ''' 
    Plot shapes given design space and latent space coordinates.
    
    Parameters:
    -----------
    Z : numpy.ndarray or None
        The latent space coordinates. If None, a grid will be generated.
    airfoils : list of Airfoil objects
        List of Airfoil objects containing the coordinates to be plotted.
    scale : float, optional
        Scaling factor for the plot.
    points_per_axis : int or None, optional
        Number of points per axis for the grid. If None, it will be calculated.
    scatter : bool, optional
        Whether to plot the shapes as a scatter plot.
    symm_axis : str or None, optional
        Axis of symmetry for the shapes.
    annotate : bool, optional
        Whether to annotate the points with their indices.
    fname : str or None, optional
        Filename to save the plot. If None, the plot will not be saved.
    **kwargs : dict
        Additional keyword arguments for plotting.
    '''
    
    plt.rc("font", size=12)
    
    if Z is None or Z.shape[1] != 2 or points_per_axis is None:
        N = len(airfoils)
        points_per_axis = int(N**.5)
        bounds = (0., 1.)
        Z = gen_grid(2, points_per_axis, bounds[0], bounds[1])  # Generate a grid
        
    scale /= points_per_axis
        
    # Create a 2D plot
    fig = plt.figure(figsize=(5, 5))
    ax = fig.add_subplot(111)
            
    for (i, z) in enumerate(Z):
        # Extract coordinates from the Airfoil object
        coordinates = airfoils[i].coordinates
        plot_shape(coordinates, z[0], z[1], ax, scale, scatter, symm_axis, **kwargs)
        if annotate:
            label = '{0}'.format(i+1)
            plt.annotate(label, xy=(z[0], z[1]), size=10)
    
    plt.xticks([])
    plt.yticks([])
    plt.axis('off')
    plt.axis('equal')
    plt.tight_layout()
    
    if fname:
        plt.savefig(fname + '.svg', dpi=600)
    plt.close()

def plot_shape(coordinates, x, y, ax, scale, scatter, symm_axis, **kwargs):
    '''
    Helper function to plot a single shape.
    
    Parameters:
    -----------
    coordinates : numpy.ndarray
        The coordinates of the shape to be plotted.
    x : float
        The x-coordinate for the shape's position.
    y : float
        The y-coordinate for the shape's position.
    ax : matplotlib.axes.Axes
        The axis to plot on.
    scale : float
        Scaling factor for the plot.
    scatter : bool
        Whether to plot the shapes as a scatter plot.
    symm_axis : str or None
        Axis of symmetry for the shapes.
    **kwargs : dict
        Additional keyword arguments for plotting.
    '''
    # Adjust coordinates based on the position (x, y) and scale
    adjusted_coords = coordinates * scale + np.array([x, y])
    
    if scatter:
        ax.scatter(adjusted_coords[:, 0], adjusted_coords[:, 1], **kwargs)
    else:
        ax.plot(adjusted_coords[:, 0], adjusted_coords[:, 1], **kwargs)
    
    if symm_axis:
        # Handle symmetry if needed
        pass

def gen_grid(dim, points_per_axis, lower_bound, upper_bound):
    '''
    Generate a grid of points in the specified dimension.
    
    Parameters:
    -----------
    dim : int
        The dimension of the grid.
    points_per_axis : int
        Number of points per axis.
    lower_bound : float
        Lower bound for the grid.
    upper_bound : float
        Upper bound for the grid.
    
    Returns:
    --------
    numpy.ndarray
        A grid of points.
    '''
    # Generate a grid of points
    grid = np.mgrid[[slice(lower_bound, upper_bound, points_per_axis*1j) for _ in range(dim)]]
    grid = grid.reshape(dim, -1).T
    return grid

In [5]:
airfoils = [Airfoil(coordinates=np.random.rand(10, 2)) for _ in range(16)]  # Example airfoils
plot_samples(None, airfoils, fname='airfoil_plot')

In [2]:
fig = go.Figure()
airfoil = Airfoil(name="NACA0012")
airfoil.draw(show=False, fig=fig, fill=False)
airfoil2 = Airfoil(name="NACA0008")
airfoil2.draw(show=True, fig=fig, fill=False, color="green")

In [3]:
kulfan = airfoil.to_kulfan_airfoil(n_weights_per_side=8)
params = kulfan.kulfan_parameters
params

{'lower_weights': array([-0.1728844 , -0.15156292, -0.17376306, -0.12768079, -0.16481846,
        -0.126376  , -0.14589789, -0.13919961]),
 'upper_weights': array([0.1728844 , 0.15156292, 0.17376306, 0.12768079, 0.16481846,
        0.126376  , 0.14589789, 0.13919961]),
 'leading_edge_weight': np.float64(3.5974444559735816e-16),
 'TE_thickness': np.float64(0.0025478997333240466)}

In [4]:
import numpy as np
from scipy.special import comb

def generate_airfoil(N1, N2, lower_weights, upper_weights, leading_edge_weight, TE_thickness, n_points_per_side):
    # Gerar pontos espaçados cosinicamente
    x = (1 - np.cos(np.linspace(0, np.pi, n_points_per_side))) / 2  

    # Função Classe
    C = (x ** N1) * ((1 - x) ** N2)

    def shape_function(w):
        # Função de forma (polinômios de Bernstein)
        N = len(w) - 1  # Ordem dos polinômios

        K = comb(N, np.arange(N + 1))  # Coeficientes binomiais de Bernstein

        dims = (len(w), len(x))

        def wide(vector):
            return np.tile(np.reshape(vector, (1, dims[1])), (dims[0], 1))

        def tall(vector):
            return np.tile(np.reshape(vector, (dims[0], 1)), (1, dims[1]))

        S_matrix = (
            tall(K)
            * wide(x) ** tall(np.arange(N + 1))
            * wide(1 - x) ** tall(N - np.arange(N + 1))
        )  # Polinômios de Bernstein multiplicados pelos pesos

        S_x = np.sum(tall(w) * S_matrix, axis=0)

        # Calcular a saída y
        y = C * S_x
        return y

    y_lower = shape_function(lower_weights)
    y_upper = shape_function(upper_weights)

    # Espessura do bordo de fuga (TE thickness)
    y_lower -= x * TE_thickness / 2
    y_upper += x * TE_thickness / 2

    # Modificação do bordo de ataque (LEM)
    y_lower += leading_edge_weight * x * (1 - x) ** (len(lower_weights) + 0.5)
    y_upper += leading_edge_weight * x * (1 - x) ** (len(upper_weights) + 0.5)

    # Criar coordenadas do aerofólio
    x = np.concatenate((x[::-1], x[1:]))
    y = np.concatenate((y_upper[::-1], y_lower[1:]))
    coordinates = np.stack((x, y), axis=1)

    return coordinates

coords = generate_airfoil(0.5, 1, params["lower_weights"], params["upper_weights"], 0, 0,150)
coords.shape

(299, 2)

In [45]:
import numpy as np
from scipy.special import comb
import tensorflow as tf
from tensorflow.keras.layers import Layer

class CSTLayer(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(CSTLayer, self).__init__(**kwargs)
        self.N1 = 0.5
        self.N2 = 1
        self.TE_thickness = 0.0
        self.leading_edge_weight = 0.0
        self.n_points_per_side = 150

    def call(self, inputs):
        # Inputs is a 3D tensor: (batch_size, 2, num_weights)
        # For example, shape: (1, 2, 12)
        batch_size = tf.shape(inputs)[0]
        num_weights = tf.shape(inputs)[2]

        # Reshape inputs to separate lower and upper weights
        # Shape: (batch_size, 2, num_weights) -> (2, batch_size * num_weights)
        inputs = tf.reshape(inputs, (batch_size * 2, num_weights))

        # Split into lower and upper weights
        lower_weights = inputs[:batch_size, :]  # Shape: (batch_size, num_weights)
        upper_weights = inputs[batch_size:, :]  # Shape: (batch_size, num_weights)

        # Generate cosinically spaced points
        x = (1 - tf.cos(tf.linspace(0.0, np.pi, self.n_points_per_side))) / 2

        # Class function
        C = (x ** self.N1) * ((1 - x) ** self.N2)

        def shape_function(w):
            # Shape function (Bernstein polynomials)
            N = tf.cast(tf.shape(w)[1] - 1, dtype=tf.float32)  # num_weights - 1

            # Bernstein binomial coefficients
            K = tf.cast(comb(N.numpy(), np.arange(N.numpy() + 1)), dtype=tf.float32)

            dims = (tf.shape(w)[1], tf.shape(x)[0])

            def wide(vector):
                return tf.tile(tf.reshape(vector, (1, dims[1])), (dims[0], 1))

            def tall(vector):
                return tf.tile(tf.reshape(vector, (dims[0], 1)), (1, dims[1]))

            S_matrix = (
                tall(K)
                * wide(x) ** tall(tf.cast(np.arange(N + 1), dtype=tf.float32))
                * wide(1 - x) ** tall(tf.cast(N - np.arange(N + 1), dtype=tf.float32))
            )  # Bernstein polynomials multiplied by weights

            S_x = tf.reduce_sum(tall(w) * S_matrix, axis=0)

            # Calculate the output y
            y = C * S_x
            return y

        # Apply shape function to lower and upper weights
        y_lower = shape_function(lower_weights)
        y_upper = shape_function(upper_weights)

        # Trailing edge thickness (TE thickness)
        y_lower -= x * self.TE_thickness / 2
        y_upper += x * self.TE_thickness / 2

        # Leading edge modification (LEM)
        y_lower += self.leading_edge_weight * x * (1 - x) ** (tf.cast(tf.shape(lower_weights)[1], dtype=tf.float32) + 0.5)
        y_upper += self.leading_edge_weight * x * (1 - x) ** (tf.cast(tf.shape(upper_weights)[1], dtype=tf.float32) + 0.5)

        # Create airfoil coordinates
        x = tf.concat([x[::-1], x[1:]], axis=0)
        y = tf.concat([y_upper[::-1], y_lower[1:]], axis=0)
        coordinates = tf.stack([x, y], axis=1)

        return coordinates

In [46]:
# Example input tensor from Conv2D layer
input_tensor = tf.random.normal((1, 2, 12))  # Shape: (1, 2, 12)

# Instantiate CSTLayer
cst_layer = CSTLayer()

# Pass input through CSTLayer
output_coordinates = cst_layer(input_tensor)

(299, 2)


In [44]:
fig = go.Figure()

airfoil = Airfoil(name="NACA8808")
airfoil.draw(fig=fig, show=False)

kulfan = airfoil.to_kulfan_airfoil(12)
np.array([kulfan.lower_weights, kulfan.upper_weights]).shape

layer = CSTLayer()
coords = layer(np.array([kulfan.lower_weights, kulfan.upper_weights]))

test = Airfoil(coordinates=coords)
test.draw(fig=fig, color="red")

In [53]:
airfoil = Airfoil("NACA0012")
airfoil.coordinates.shape
airfoil = airfoil.repanel(75)
airfoil.coordinates.shape

(149, 2)

In [ ]:
fig = go.Figure()
airfoil = Airfoil(name="NACA4412")
airfoil.draw(show=False, fig=fig, fill=False)
random_kulfan = KulfanAirfoil(lower_weights=np.array([1, 1]), upper_weights=np.array([0.2,0.2]), leading_edge_weight=0, TE_thickness=0, N1=0.5, N2=1)
random_kulfan.draw(fig=fig, show=True, fill=False, color="green")

In [ ]:
from src import run_xfoil
output = run_xfoil(airfoil, 0, 80, 2)
output

In [ ]:
output = airfoil.generate_polars(alpha_i=-5, alpha_f=20, alpha_step=0.25)

In [ ]:
airfoil.plot_polars()

In [ ]:
airfoil.polars[0]

In [3]:
airfoil_database_path = asb._asb_root / "geometry" / "airfoil" / "airfoil_database"

UIUC_airfoils = [
    asb.Airfoil(name=filename.stem).normalize()
    for filename in airfoil_database_path.iterdir() if filename.suffix == ".dat"
]

In [11]:
rand_airfoil = lambda: np.random.choice(UIUC_airfoils)
airfoil = rand_airfoil()

In [ ]:
airfoil.generate_polars()
airfoil.plot_polars()